In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
print(os.getcwd())
# 更改当前路径到指定目录
os.chdir('/content/drive/MyDrive/Defect-detection/code')

Mounted at /content/drive
/content


In [9]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
gpu_num = torch.cuda.device_count()
print(gpu_num)
print(torch.cuda.get_device_name())

cuda
1
NVIDIA A100-SXM4-40GB


In [11]:
import os
print(os.getcwd())
# 更改当前路径到指定目录
os.chdir('/content/drive/MyDrive/Defect-detection/code')

/content/drive/MyDrive/Defect-detection/code


In [12]:
!ls

 CodeBERT       saved_models_All			  saved_models_merged
 model.py       saved_models_All_Transformed_Data	  saved_models_ori
 __pycache__   'saved_models_All_Transformed_Data (1)'	  saved_models_small
 run_data.py    saved_models_All_Transformed_Data_1	  save_models_small
 run.py        'saved_models_All_Transformed_Data (2)'	  test.log
 run_small.py  'saved_models_All_Transformed_Data (3)'	  train.lo
 saved_models   saved_models_All_Transformed_Data_final   We_saved_models


**Fine-tune**

In [ ]:
!python run_small.py \
    --output_dir=./We_saved_models \
    --model_type=roberta \
    --tokenizer_name=microsoft/codebert-base \
    --model_name_or_path=microsoft/codebert-base \
    --do_train \
    --train_data_file=../dataset/trainset.jsonl \
    --eval_data_file=../dataset/valid.jsonl \
    --test_data_file=../dataset/test.jsonl \
    --epoch 5 \
    --block_size 256 \
    --train_batch_size 32 \
    --eval_batch_size 64 \
    --learning_rate 2e-5 \
    --max_grad_norm 1.0 \
    --evaluate_during_training \
    --seed 123456  2>&1 | tee train.lo

2023-05-31 03:25:49.335983: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-31 03:25:49.390278: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-31 03:25:50.315429: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
05/31/2023 03:25:52 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
Some weights of the model checkpoint at microsoft/codebert-base were not used when initializing RobertaForSequ

**Test**

In [ ]:
!python run_small.py \
    --output_dir=./We_saved_models \
    --model_type=roberta \
    --tokenizer_name=microsoft/codebert-base \
    --model_name_or_path=microsoft/codebert-base \
    --do_eval \
    --do_test \
    --train_data_file=../dataset/trainset.jsonl \
    --eval_data_file=../dataset/valid.jsonl \
    --test_data_file=../dataset/test.jsonl \
    --epoch 5 \
    --block_size 400 \
    --train_batch_size 32 \
    --eval_batch_size 64 \
    --learning_rate 2e-5 \
    --max_grad_norm 1.0 \
    --evaluate_during_training \
    --seed 123456 2>&1 | tee test.log

**Show result**

In [ ]:
!python ../evaluator/evaluator.py -a ../dataset/test.jsonl -p We_saved_models/predictions.txt